In [253]:
# From the Fortress doc: Comprehending List Comprehensions

# A Pythagorean triple is three integers a, b, and c representing the sides of a right triangle with hypotenuse c.
# This means that a**2 + b**2 = c**2, which we'll refer to as the Pythagorean equality.
# We will require that all three sides are less than some specified maximum value m, and for uniqueness
# we'll also require that a < b < c < m. Also, triples should be primitive — not a multiple of another triple.
# Much more information on Pythagorean Triples can be found in this Wikipedia article:
# https://en.wikipedia.org/wiki/Pythagorean_triple

# Define two helper functions, coprime and even, and a trip object.
import math
import functools

def even(x): return 0 == x%2
print([even(i) for i in range(-3,5)])

def coprime (x,y): return 1 == math.gcd (x,y)
print ([coprime(3,i) for i in range(-1,10)])

class Trip():
    def __init__(self,a,b,c):
        self.a, self.b, self.c = a, b, c
        assert self.c**2 == self.a**2 + self.b**2, f"Bad Trip: <{self.a} {self.b} {self.c}>"
    
    def __eq__(self,t): return self.a == t.a and self.b == t.b and self.c == t.c
    
    def __repr__(self): return f"Trip: <{self.a} {self.b} {self.c}>"

print(Trip(3,4,5))
Trip(3,4,6)

[False, True, False, True, False, True, False, True]
[True, False, True, True, False, True, True, False, True, True, False]
Trip: <3 4 5>


AssertionError: Bad Trip: <3 4 6>

In [254]:
# We are looking to generate lists like this one, from Wikipedia:
Wikipedia100 = [Trip( 3, 4, 5), Trip( 5,12,13), Trip( 7,24,25),
                Trip( 8,15,17), Trip( 9,40,41), Trip(11,60,61),
                Trip(12,35,37), Trip(13,84,85), Trip(16,63,65),
                Trip(20,21,29), Trip(28,45,53), Trip(33,56,65),
                Trip(36,77,85), Trip(39,80,89), Trip(48,55,73),
                Trip(65,72,97)
                ]

print(Wikipedia100)

[Trip: <3 4 5>, Trip: <5 12 13>, Trip: <7 24 25>, Trip: <8 15 17>, Trip: <9 40 41>, Trip: <11 60 61>, Trip: <12 35 37>, Trip: <13 84 85>, Trip: <16 63 65>, Trip: <20 21 29>, Trip: <28 45 53>, Trip: <33 56 65>, Trip: <36 77 85>, Trip: <39 80 89>, Trip: <48 55 73>, Trip: <65 72 97>]


In [259]:
# Here's a direct enumerate-and-filter specification, parametrized by m, the maximum side length.
m = 100
trips1 = [Trip(a,b,int(c))
          for a in range(1,m+1)
          for b in range(1,m+1)
          for c in range(1,m+1)
          if a < b < c
          and a**2 + b**2 == c**2
          and coprime(a,b)
       ]

def check_trips (trips):
    assert(all ([w == t for w,t in zip (Wikipedia100, trips)])), "Trips don't match: " + print (trips)
    print("Good Trips")

check_trips(trips1)

Good Trips


In [239]:
# The expression: for a in range(1,m+1) for b in range(1,m+1) for c in range(1,m+1) is like a triply nested loop.
# In this case, the order of the generators is irrelevant (and the compiler might leverage this), but generally
# the order matters as we'll see. We also specified three filters: an ordering of a, b, and c, the Pythagorean
# equality, and coprime (a,b), which ensures that the triple is primitive.
#
# We can fold the first filter, a < b < c, into the lower and upper bounds of the generators.
trips2 = [Trip(a,b,int(c))
          for a in range(1,m-1)
          for b in range(a+1,m)
          for c in range(b+1,m+1)
          if a**2 + b**2 == c**2
          and coprime(a,b)
       ]
check_trips(trips2)

Good Trips


In [247]:
# This sort of transformation is known as filter promotion. Rather than generate and then filter, we can
# avoid generating some values in the first place. We can imagine that that a compiler, a run time
# system, or in Fortress' case, a clever library, could accomplish simple filter promotion. Now c
# generation must come after b generation which in turn must come after a generation as a is used in
# the bounds of b generation which in turn is used in the bounds of c generation --- the generators can
# no longer be freely interchanged --- c generation is "more inner" than b generation which in turn is
# "more inner" than a generation.
#
# Shouldn't this sort of transformation be the programmer's responsibility? Perhaps, but sometime in
# mathematics we specify sets this way, and it can be very concise and easy to understand.
#
# What about the next filter, the Pythagorean equality? Once a and b are selected, this filter tells us
# there is at most one one possible value to consider for c, we have a candidate if c is an integer. First
# rewrite the filter c = SQRT(a**2 + b**2). If that produces an integer we have a candidate; otherwise,
# we're not interested.
#
# There's also a bit of housekeeping, as c must satisfy the generator bounds, ie, b+1 <= c <= m. The
# lower bound comes for free, but we have to check the upper bound explicitly with a filter: c <= m.

trips3 = [Trip(a,b,int(c))
          for a in range(1,m-1)
          for b in range(a+1,m)
          for c in [math.sqrt(a**2 + b**2)]
          if c == int(c) and c <= m
          and coprime(a,b)
         ]
check_trips(trips3)

Good Trips


In [248]:
# Can we promote the filter c <= m? It can be absorbed into b generation. Since c**2 = a**2 + b**2 and
# c <= m, a**2 + b**2 <= m**2, giving us b <= SQRT(m**2 - a**2). This only produces values at most m-1.

trips4 = [Trip(a,b,int(c))
          for a in range(1,m-1)
          for b in range(a+1,int(math.sqrt(m**2 - a**2)))
          for c in [math.sqrt(a**2 + b**2)]
          if c == int(c) and c <= m
          and coprime(a,b)
         ]
check_trips(trips4)

Good Trips


In [249]:
# It would be pretty impressive if a library or compiler could automatically do the filter promotion we've
# done by hand:
#  1. Fold a < b < c into b and c generation. This is the easiest one.
#  2. Calculate c rather than generate and filter. This involves quadratic algebraic manipulation.
#  3. Fold c < m into b generation (upper bound): again, algebraic manipulation.
#
# It's tricky, but all of the preceding and the following could conceivably be produced automatically by a
# clever enough system, perhaps an MS project, or an undergrad capstone project for a very clever undergrad.
# Just for fun, let's pull common divisibility by two out of the coprime filter.

trips5 = [Trip(a,b,int(c))
          for a in range(1,m-1)
          for b in range(a+1,int(math.sqrt(m**2 - a**2)))
          for c in [math.sqrt(a**2 + b**2)]
          if c == int(c) and c <= m
          and not (even(a) and even(b))
          and coprime(a,b)
         ]
check_trips(trips5)

Good Trips


In [250]:
# Then fold that into b generation using striding.

trips6 = [Trip(a,b,int(c))
          for a in range(1,m-1)
          for b in range(a+1,int(math.sqrt(m**2 - a**2)),2 if even(a) else 1)
          for c in [math.sqrt(a**2 + b**2)]
          if c == int(c) and c <= m
          and not (even(a) and even(b))
          and coprime(a,b)
        ]
check_trips(trips6)

Good Trips


In [251]:
# These last two steps could conceivably be done automagically, but that would take a really clever researcher.
# Is there more fun to be had? It turns out that exactly one of a and b are even, but proving that is
# reasonably tricky. However, given that fact, we can write the stronger guard (even a) XOR (even b),
# or, we could fold that clever fact into simplified striding in b generation.

trips7 = [Trip(a,b,int(c))
          for a in range(1,m-1)
          for b in range(a+1,int(math.sqrt(m**2 - a**2)),2)
          for c in [math.sqrt(a**2 + b**2)]
          if c == int(c) and c <= m
          and coprime(a,b)
        ]
check_trips(trips7)

Good Trips


In [252]:
# And, it also turns out that the smallest leg in a triple is 3.

trips8 = [Trip(a,b,int(c))
          for a in range(3,m-1)
          for b in range(a+1,int(math.sqrt(m**2 - a**2)),2)
          for c in [math.sqrt(a**2 + b**2)]
          if c == int(c) and c <= m
          and coprime(a,b)
        ]
check_trips(trips8)

Good Trips


In [246]:
# Performing these last three transformations automatically would also be impressive research.
#  1. & 2. Turn common divisibility by 2 into b striding (1: complex striding, and 2: simple striding)
#  3. Start a with 3. Good luck with this one. :-)
